In [14]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# sys.path.append(os.path.abspath('../'))


# from src.data_preprocessing import (
#     load_data,
#     handle_missing_values,
#     clean_data,
#     exploratory_data_analysis,
#     merge_datasets,
#     feature_engineering,
#     normalize_and_scale,
#     encode_categorical_features,
#     save_processed_data
# )

# # Load data
# fraud_data, ip_data = load_data()

# # Handle Missing Values
# fraud_data = handle_missing_values(fraud_data)

# # Data Cleaning
# fraud_data = clean_data(fraud_data)

# # Exploratory Data Analysis (EDA)
# exploratory_data_analysis(fraud_data)

# # Merge Datasets for Geolocation Analysis
# merged_data = merge_datasets(fraud_data, ip_data)

# # Feature Engineering
# merged_data = feature_engineering(merged_data)

# # Normalization and Scaling
# merged_data = normalize_and_scale(merged_data)

# # Encode Categorical Features
# final_data = encode_categorical_features(merged_data)

# # Save the cleaned and processed data
# save_processed_data(final_data, 'data/processed/processed_Fraud_Data.csv')

In [15]:
fraud_data =pd.read_csv("../data/raw/Fraud_Data.csv")

In [16]:
ip_data= pd.read_csv("../data/raw/IpAddress_to_Country.csv")

In [17]:
credit_data=pd.read_csv("../data/raw/creditcard.csv")

In [13]:
fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [18]:
ip_data.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


In [19]:
credit_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [27]:
print("Data Types of fraud Data\n\n" ,fraud_data.dtypes)
print("Data Types of Ip Data\n\n" ,ip_data.dtypes)
print("Data Types of credit Data\n\n" ,credit_data.dtypes)

Data Types of fraud Data

 user_id             int64
signup_time        object
purchase_time      object
purchase_value      int64
device_id          object
source             object
browser            object
sex                object
age                 int64
ip_address        float64
class               int64
dtype: object
Data Types of Ip Data

 lower_bound_ip_address    float64
upper_bound_ip_address      int64
country                    object
dtype: object
Data Types of credit Data

 Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V

In [31]:
print("-----------------------Null value count fro fraud data\n\n",fraud_data.isnull().count())
print("++++++++++++++++++++++++++++Null value count for ip data\n\n",ip_data.isnull().count())
print("_____________________Null value count for credit data\n\n",credit_data.isnull().count())

-----------------------Null value count fro fraud data

 user_id           151112
signup_time       151112
purchase_time     151112
purchase_value    151112
device_id         151112
source            151112
browser           151112
sex               151112
age               151112
ip_address        151112
class             151112
dtype: int64
++++++++++++++++++++++++++++Null value count for ip data

 lower_bound_ip_address    138846
upper_bound_ip_address    138846
country                   138846
dtype: int64
_____________________Null value count for credit data

 Time      284807
V1        284807
V2        284807
V3        284807
V4        284807
V5        284807
V6        284807
V7        284807
V8        284807
V9        284807
V10       284807
V11       284807
V12       284807
V13       284807
V14       284807
V15       284807
V16       284807
V17       284807
V18       284807
V19       284807
V20       284807
V21       284807
V22       284807
V23       284807
V24       284807
V25